In [1]:
#Runs the strategy over a 1, 2, 5, 10, and 20 year period
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

end_date = '2025-08-08'
end_dt = datetime.strptime(end_date, '%Y-%m-%d')

periods = {
    '1_year': end_dt - timedelta(days = 365),
    '2_years': end_dt - timedelta(days = 365*2),
    '5_years': end_dt - timedelta(days = 365*5),
    '10_years': end_dt - timedelta(days = 365*10),
    '20_years': end_dt - timedelta(days = 365*20)
}

full_data = yf.download("^GSPC", start = "1990-01-01", end = end_date, auto_adjust = True)

def run_strategy(dataF, start_date, period_name):
    buffer_days = 200
    buffer_start = start_date - timedelta(days=buffer_days)
    
    period_data = dataF[dataF.index >= buffer_start].copy()
    period_data['ma50'] = period_data['Close'].rolling(50).mean()
    period_data['ma200'] = period_data['Close'].rolling(200).mean()

    period_data = period_data[period_data.index >= start_date]
    
    def signal_generator(df):
        m50 = df.ma50.iloc[-1]
        m200 = df.ma200.iloc[-1]
        previous_m50 = df.ma50.iloc[-2]
        previous_m200 = df.ma200.iloc[-2]
        
        if (previous_m50 > previous_m200 and m200 > m50):
            return 1 
        elif (previous_m50 < previous_m200 and m200 < m50):
            return 2  
        else:
            return 0
    
    signal = [0]
    for i in range(1, len(period_data)):
        df = period_data.iloc[i-1:i+1]
        signal.append(signal_generator(df))
    period_data["signal"] = signal
    
    position = 0
    entry_price = 0
    position_value = 0
    cash = 1000
    portfolio_value = [cash]
    trades = []
    
    for i in range(1, len(period_data)):
        current_price = period_data.Close.iloc[i]
        current_signal = period_data.signal.iloc[i]
        if position == 1 and current_signal == 2:
            pnl = 1000 * (entry_price/current_price - 1)
            cash += pnl
            trades.append(('Close Short', period_data.index[i], current_price, pnl))
            position = 2
            entry_price = current_price
            trades.append(('Open Long', period_data.index[i], current_price, 0))
        elif position == 2 and current_signal == 1:
            pnl = 1000 * (current_price/entry_price - 1)
            cash += pnl
            trades.append(('Close Long', period_data.index[i], current_price, pnl))
            position = 1
            entry_price = current_price
            trades.append(('Open Short', period_data.index[i], current_price, 0))
        elif position == 0:
            if current_signal == 1:
                position = 1
                entry_price = current_price
                cash -= 1000
                trades.append(('Open Short', period_data.index[i], current_price, 0))
            elif current_signal == 2:
                position = 2
                entry_price = current_price
                cash -= 1000
                trades.append(('Open Long', period_data.index[i], current_price, 0))
        if position == 1:
            position_value = 1000 * (entry_price/current_price)
        elif position == 2:
            position_value = 1000 * (current_price/entry_price)
        else:
            position_value = 0
        
        portfolio_value.append(cash + position_value)
    
    period_data['portfolio_value'] = portfolio_value[:len(period_data)]
    final_value = float(period_data['portfolio_value'].iloc[-1].item())
    total_return = (final_value - 1000) / 1000 * 100
    
    results = period_data[['Close', 'signal', 'portfolio_value']]
    results.to_csv(f'strategy_results_{period_name}.csv', index=True)
    
    return {
        'period': period_name,
        'start_date': start_date.strftime('%Y-%m-%d'),
        'end_date': end_date,
        'final_value': final_value,
        'total_return': total_return,
        'num_trades': (len(trades)+1)//2
    }

performance_results = []
for period_name, start_date in periods.items():
    print(f"\nRunning strategy for {period_name} period...")
    result = run_strategy(full_data, start_date, period_name)
    performance_results.append(result)
    print(f"Completed {period_name}: {result['total_return']:.2f}% return")

summary_df = pd.DataFrame(performance_results)
print("\nPerformance Summary:")
print(summary_df[['period', 'start_date', 'end_date', 'final_value', 'total_return', 'num_trades']])

#summary_df.to_csv('strategy_performance_summary.csv', index = False)

[*********************100%***********************]  1 of 1 completed



Running strategy for 1_year period...
Completed 1_year: -10.49% return

Running strategy for 2_years period...
Completed 2_years: -10.49% return

Running strategy for 5_years period...
Completed 5_years: 18.69% return

Running strategy for 10_years period...
Completed 10_years: 31.29% return

Running strategy for 20_years period...
Completed 20_years: 146.23% return

Performance Summary:
     period  start_date    end_date  final_value  total_return  num_trades
0    1_year  2024-08-08  2025-08-08   895.119669    -10.488033           2
1   2_years  2023-08-09  2025-08-08   895.119669    -10.488033           2
2   5_years  2020-08-09  2025-08-08  1186.897334     18.689733           4
3  10_years  2015-08-11  2025-08-08  1312.943045     31.294305          11
4  20_years  2005-08-13  2025-08-08  2462.261245    146.226124          20
